https://data-ogauthority.opendata.arcgis.com/search?tags=Zip  
https://data-ogauthority.opendata.arcgis.com/datasets/oga-offshore-zipped-shapefiles-wgs84  
https://dataogauthority.blob.core.windows.net/external/OpenDataZips/OGA_OFF_WGS84.zip  


#### map tiles

https://en.wikipedia.org/wiki/Vector_tiles  
https://openmaptiles.com/downloads/planet/  

In [35]:

import geopandas
import ipyleaflet
import json
import numpy as np
import pandas as pd

from dateutil import parser as duparser

In [2]:
oga_pl = pd.read_csv("DealPipelinesKis.csv")

In [3]:
pldf = geopandas.read_file("OGA_OFF_WGS84/OGA_Pipelines_WGS84.shp")

In [4]:
pldf.head()

,OBJECTID,LICGEOE_ID,BLOCK,REASON,MOD_NOTE,NOTE,Shape_STAr,Shape_STLe,ID,NAME,...,CURR_LIC_6,OGA_COP,PIPE_DTINO,PIPE_NAME,DIAMETER,UNITS,FLUID,EXPOSED,UNTRENCHED,geometry
0,1,None,None,None,None,None,None,15514.313888,500003635,None,...,None,None,None,CLIPPER PM TO BARQUE PL CABLE,2.0,inch,None,None,None,"LINESTRING (1.73245 53.45873, 1.73342 53.45938..."
1,2,None,None,None,None,None,None,2916.737757,500003636,None,...,None,None,None,LEMAN BP TO LEMAN E,1.0,inch,None,None,None,"LINESTRING (2.18466 53.07610, 2.18408 53.07649..."
2,3,None,None,None,None,None,None,47773.373769,500003637,None,...,None,None,PL934,NELSON TO FULMAR RECEIVER TEE,10.0,inch,GAS,None,None,"LINESTRING (1.14443 57.66231, 1.14477 57.66234..."
3,4,None,None,None,None,None,None,32025.592920,500003780,None,...,None,None,PL1030,DD-POA GAS EXPORT,20.0,inch,GAS,Y,None,"LINESTRING (-3.34012 53.35390, -3.38517 53.379..."
4,5,None,None,None,None,None,None,33921.405183,500003781,None,...,None,None,PL931,ORWELL TO THAMES RA,16.0,inch,GAS,None,None,"LINESTRING (2.54646 53.08393, 2.56516 53.08413..."


In [5]:
PL1844 = pldf.loc[pldf['PIPE_DTINO'] == "PL1844"]
PL1844

,OBJECTID,LICGEOE_ID,BLOCK,REASON,MOD_NOTE,NOTE,Shape_STAr,Shape_STLe,ID,NAME,...,CURR_LIC_6,OGA_COP,PIPE_DTINO,PIPE_NAME,DIAMETER,UNITS,FLUID,EXPOSED,UNTRENCHED,geometry
3430,3431,None,None,None,None,None,None,105206.633615,500005907,None,...,None,None,PL1844,CLAIR TO SULLOM VOE,22.0,inch,OIL,None,None,"LINESTRING (-2.54273 60.69127, -2.37246 60.699..."


In [ ]:
#countries_gdf.to_file("countries.geojson", driver='GeoJSON')
PL1844["geometry"].to_file("PL1844.geojson", driver='GeoJSON')

In [40]:
PL1844_json = pldf.to_json() #PL1844.to_json()
data = json.loads( PL1844_json )
#dd

In [41]:
# remove empty properties data
for _f in data['features']:
    _meta = {k:v for k,v in _f['properties'].items() if v is not None} #_f['properties'].copy()
    _id = _f['properties']['ID']
    _ogapl = oga_pl.loc[oga_pl['PIPELINEID'] == _id]
    if not _ogapl.empty:
        dd = _ogapl.iloc[0].to_dict()
        dd = {k:v for k,v in dd.items() if pd.notna(v) and str(v).strip() != "" }
        _meta.update(dd)

    new_meta = {}
    if 'PIPELINE_NAME' in _meta:
        new_meta["name"] = _meta['PIPELINE_NAME']
    elif 'PIPE_NAME' in _meta:
        new_meta["name"] = _meta['PIPE_NAME']
    elif 'PIPELINE_STDNAME' in _meta:
        new_meta["name"] = _meta['PIPELINE_STDNAME']
        
    if 'OPERATOR' in _meta:
        new_meta["operator"] = _meta['OPERATOR']
    elif 'PIPE_NAME' in _meta:
        new_meta["operator"] = _meta['NAMING_COMPANY']  

    if 'PIPELINE_DTINO' in _meta:
        new_meta["DTINO"] = _meta['PIPELINE_DTINO']
    elif 'PIPE_DTINO' in _meta:
        new_meta["DTINO"] = _meta['PIPE_DTINO']

    if 'DESCRIPTION' in _meta:
        new_meta["description"] = _meta['DESCRIPTION']
    elif 'DESC_' in _meta:
        new_meta["description"] = _meta['DESC_']

    if 'ID' in _meta:
        new_meta["OGA_db_id"] = _meta['ID']
    elif 'PIPELINEID' in _meta:
        new_meta["OGA_db_id"] = _meta['PIPELINEID']
    elif 'EntityKey' in _meta:
        new_meta["OGA_db_id"] = _meta['EntityKey']

    if 'START_DATE' in _meta:
        new_meta["startup_date"] = duparser.parse(_meta['START_DATE']).date().isoformat()

    if 'PIPELINE_LENGTH_M' in _meta:
        new_meta["length"] = _meta['PIPELINE_LENGTH_M']

    if 'FLUID' in _meta:
        new_meta["content"] = _meta['FLUID']
    elif 'FLUID_CONVEYED' in _meta:
        new_meta["content"] = _meta['FLUID_CONVEYED']    

    if 'DIAMETER' in _meta:
        new_meta["D_nom"] = _meta['DIAMETER']
        new_meta["Do"] = _meta['DIAMETER']
        if ('UNITS' in _meta and _meta['UNITS'].lower()=='inch' or
           'DIAMETER_UNITS' in _meta and _meta['DIAMETER_UNITS'].lower()=='inch'):
            new_meta["Do"] = _meta['DIAMETER']*0.0254

    if 'STATUS' in _meta:
        new_meta["status"] = _meta['STATUS']

    if 'COMMENTS' in _meta:
        new_meta["comments"] = _meta['COMMENTS']        

    if 'INST_TYPE' in _meta:
        new_meta["INST_TYPE"] = _meta['INST_TYPE']         

    if 'INSULATION_COATING_TYPE' in _meta:
        new_meta["INSULATION_COATING_TYPE"] = _meta['INSULATION_COATING_TYPE']           

    if 'INTERNAL_DIAMETER_MM' in _meta:
        new_meta["Di"] = _meta['INTERNAL_DIAMETER_MM'] / 1000.0        

    if 'WALL_THICKNESS_MM' in _meta:
        new_meta["WT"] = _meta['WALL_THICKNESS_MM'] / 1000.0           
        
    if 'OPERATING_PRESSURE_MAX_BARG' in _meta:
        new_meta["OPERATING_PRESSURE_MAX_BARG"] = _meta['OPERATING_PRESSURE_MAX_BARG']        
        
    _f['properties'] = new_meta.copy()
        #_f['properties'] = {k:v for k,v in _meta.items() if v is not None}
_meta #_f['properties']

{'DESC_': 'CULZEAN FSO (AILSA) - ANCHOR LINE 3',
 'DIAMETER': 0.0,
 'ID': 510001771,
 'INS_DATE': '20170421',
 'OBJECTID': 3476,
 'OPERATOR': 'TOTAL',
 'PIPE_NAME': 'CULZEAN FSO (AILSA) - ANCHOR LINE 3',
 'STATUS': 'ACTIVE',
 'Shape_STLe': 172.03062283728121,
 'UPD_DATE': '20190502',
 'PIPELINE_NAME': 'CULZEAN FSO (AILSA) - ANCHOR LINE 3',
 'INST_TYPE': 'ANCHOR CABLE',
 'DESCRIPTION': 'CULZEAN FSO (AILSA) - ANCHOR LINE 3',
 'COMMENTS': 'Partially installed on seabed but FSO not yet on locationPartially installed on seabed but FSO not yet on locationDue to be installed later in 2017',
 'DATA_SOURCE': 'MAERSK',
 'KP_ORIGIN_CRS': 23031.0,
 'KP_ORIGIN_CRS_DESC': 'ED_1950_UTM_Zone_31N',
 'KP_ORIGIN_DESC': 'Culzean FSO Turret Centre',
 'KP_ORIGIN_X_LONG': 437309.2,
 'KP_ORIGIN_Y_LAT': 6339837.5,
 'NAMING_COMPANY': 'MAERSK',
 'VERIFIED': 'Y',
 'QC_FLAG': 'Y',
 'QC_ISSUE': 35.0,
 'QC_COMMENTS': 'UPDATED V38 - TRANS V37 -UPDATED V35 - UPDATED V35 - ADDED V34',
 'DATE_ENTERED': '21-Apr-2017',
 '

{'name': 'CLAIR TO SULLOM VOE',
 'operator': 'BP',
 'DTINO': 'PL1844',
 'description': 'CLAIR TO SULLOM VOE 22IN OIL EXPORT LINE',
 'OGA_db_id': 500005907,
 'startup_date': '2003-07-01',
 'length': 105200.86451752801,
 'content': 'OIL',
 'D_nom': 22.0,
 'Do': 0.5588,
 'status': 'ACTIVE'}

In [42]:
#pldf.to_file("OGA_pipelines.geojson", driver='GeoJSON')
with open("OGA_pipelines.geojson", 'w') as fh:
    json.dump(data, fh)

In [15]:
_ogapl.empty

False

In [ ]:
pass

In [43]:
lmap = ipyleaflet.Map(
    basemap=ipyleaflet.basemaps.Esri.WorldImagery,
    center=(59.0, 2.0),
    zoom=6
)

lmap

Map(center=[59.0, 2.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

In [45]:
geo_json = ipyleaflet.GeoJSON(data=data, style = {'color': 'red', 'opacity':1, 'weight':1.9, 'fillOpacity':0.1})
lmap.add_layer(geo_json)